In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import time
from bs4 import BeautifulSoup, Comment
from pathlib import Path
from tqdm import tqdm
%matplotlib inline

plt.style.use('fivethirtyeight')
sns.set_context('notebook')

# Web Scraping

https://sofifa.com

In [2]:
# referenced from https://realpython.com/python-web-scraping-practical-introduction/
def simple_get(url):
    """
    Attempts to scrape the content at 'url' by making a HTTP GET request. 
    If the content-type of the response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    import requests
    from contextlib import closing
    
    try:
        with closing(requests.get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except AssertionError as error:
        print(error)
        print('Error in scraping of url')


def is_good_response(resp):
    """
    Returns True if response is some kind of HTML/XML
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
           and content_type is not None
           and content_type.find('html') > -1)


We note that the information on the website is categorized by FIFA update dates. For each FIFA update date, we have different national teams and national players with different statistics. We note that there does not seem to be a strict order to the update dates. Some months have several updates, while others have only 1 update. We have all the data starting from August 2006.

In [3]:
def get_date_href():
    url = 'https://sofifa.com/'
    html = BeautifulSoup(simple_get(url), 'html.parser')
    
    dates_href = {}
    
    # for each month
    for m in html.find_all('div', attrs={'class': 'card-title h5'})[:-8]: # note the last 8 items are not dates
        month = m.get_text()
        
        # for each day of the month
        for d in m.find_next('div').find_all('a'):
            day = d.get_text()
            date = day+' '+month
            href = d.get('href')
            
            dates_href[date] = href
    
    return dates_href

In [4]:
dates_href = get_date_href()

In [5]:
# Find all the href attributes for each national team

def get_nation_href(date_url):
    """Given a date url corresponding to a specific update of FIFA 18, returns a dictionary 
    containing the teams and their hrefs"""
    url = 'https://sofifa.com/teams/national'+date_url   
    html = BeautifulSoup(simple_get(url), 'html.parser')

    teams_href = {}

    for link in html.find_all('a', attrs={'href': re.compile("^/team/.+")}):
        if link.get_text() not in teams_href:
            teams_href[link.get_text()] = link.get('href')
            
    return teams_href

In [6]:
teams_href = get_nation_href('?v=WC18&e=159126&set=true') # corresponds to FIFA WC18 Expansion Jun 16
len(teams_href)

49

We note that at different dates, there may be different teams represented in the game. As such, we loop through all the possible dates to find the full list of all teams that have ever been represented in the game.

In [7]:
full_teams_href = {}

for d, dhref in dates_href.items():    
    teams_href = get_nation_href(dhref)
    # merges dictionaries
    full_teams_href = {**teams_href, **full_teams_href}

In [8]:
len(full_teams_href)

62

We now have the urls of all 62 national teams that has ever appeared in the game. We note that the current urls link to the most recent ratings of the teams. We can find the urls of all players in the national teams from this page as well. We now wish to collect team specific data for all these teams across all the different dates.

In [65]:
url_team = 'https://sofifa.com/team/1337/germany/'
html_team = BeautifulSoup(simple_get(url_team), 'html.parser')
html_team.find(string='Whole Team Average Age').next_element

'25.35'

In [22]:
team_stats = {}
for date, dhref in tqdm.tqdm(dates_href.items()):
    for team, thref in full_teams_href.items():
        url_team = 'https://sofifa.com'+thref+dhref
        html_team = BeautifulSoup(simple_get(url_team), 'html.parser')
        
        span = html_team.find('div', attrs={'class': 'card-body stats'}).find_all('span')
        span_label = (html_team
                      .find('div', attrs={'class': 'card-body stats'})
                      .find_all('span', attrs={'class': re.compile("label.+")}))
        
        overall = float(span_label[0].get_text())
        attack = float(span_label[1].get_text())
        midfield = float(span_label[2].get_text())
        defence = float(span_label[3].get_text())

        prestige = float(html_team.find(string='International Prestige').find_next('span').get_text())
        start_age = float(html_team.find(string='Starting XI Average Age').next_element)
        full_age = float(html_team.find(string='Whole Team Average Age').next_element)
        
        team_stats[(team, date)] = {
            'overall': overall,
            'attack': attack,
            'midfield': midfield,
            'defence': defence,
            'prestige': prestige,
            'start_age': start_age,
            'full_age': full_age
        }
    


 16%|███████████▉                                                                 | 67/432 [1:25:41<8:00:04, 78.92s/it]


 31%|███████████████████████▊                                                    | 135/432 [2:46:49<6:59:35, 84.77s/it]


 47%|███████████████████████████████████▋                                        | 203/432 [4:12:22<4:49:56, 75.97s/it]


 63%|███████████████████████████████████████████████▋                            | 271/432 [5:26:12<2:56:54, 65.93s/it]


 78%|███████████████████████████████████████████████████████████▋                | 339/432 [6:40:24<1:28:55, 57.37s/it]


 94%|█████████████████████████████████████████████████████████████████████████▍    | 407/432 [7:55:54<35:13, 84.54s/it]


100%|██████████████████████████████████████████████████████████████████████████████| 432/432 [8:34:32<00:00, 88.43s/it]


In [56]:
team_stats_df = pd.DataFrame(team_stats).T.reset_index()
team_stats_df.columns = ['team', 'date', 'attack', 'defence', 'full_age', 
                         'midfield', 'overall', 'prestige', 'start_age']

In [58]:
team_stats_df.to_csv('../datasets/sofifa/team_stats.csv',index=False)

We could also obtain the player data for each of the national teams at each of the dates represented. However, we first write a function to find all the player information for a given national team at a given point in time.

In [59]:
def get_players_href(team_href, date_url):
    """Given a team href and a segment of a date url, corresponding to a specific update of FIFA 18
    returns a list of all players in the team, and a list of all 
    players href in the team, for all teams"""
    url_team = 'https://sofifa.com'+team_href+date_url 
    html_team = BeautifulSoup(simple_get(url_team), 'html.parser')

    player_href = {}
    for link in html_team.find_all('a', attrs={'href': re.compile("^/player/.+")}):
        if link.get_text() not in player_href:
            player_href[link.get_text()] = link.get('href')
    
    return player_href

We now have the players href for a national team at a given point in time. We now have to obtain all the information for each of the players from these hrefs.

In [60]:
def get_players(player_href, date_url):
    """Given a players href,  and a segment of a date url, corresponding to a specific update of FIFA 18,
    returns the summary data for that player"""

    url_player = 'https://sofifa.com'+player_href+date_url
    html_player = BeautifulSoup(simple_get(url_player), 'html.parser')

    h1 = html_player.find('h1')

    name_id = h1.get_text()
    name = name_id.split('(')[0]
    ID = re.search(r'\((.+)\)', name_id).group(1).split(' ')[-1]
    full_name = h1.find_next('div').next_element.strip()
    
    # account for multiple preferred positions
    position = ''
    for p in h1.find_next('div').find_all('span'):
        pos = p.get_text()+' '
        position += pos
    position.strip()

    age = re.search(r'Age (\d+) .+' , h1.find_next('div').find_all('span')[-1].next_sibling).group(1)
            
    # account for +/- signs
    span = html_player.find('div', attrs={'class': 'card-body stats'}).find_all('span')
    span_label = (html_player
                  .find('div', attrs={'class': 'card-body stats'})
                  .find_all('span', attrs={'class': re.compile("label.+")}))
            
    overall = float(span_label[0].get_text())
    potential = float(span_label[1].get_text())
    value = span[-2].get_text()
    wage = span[-1].get_text()
    
    # account for different order of clubs and national teams
    club_nation = html_player.find_all('figure', attrs={'class': "avatar avatar-sm"})
    cn0 = club_nation[0].find_next('a', attrs={'href': re.compile("^/team/.*")})
    if len(club_nation) > 1:
        cn1 = club_nation[1].find_next('a', attrs={'href': re.compile("^/team/.*")})
        if cn0.get_text() == tname:
            club = cn1.get_text()
            club_rating = float(cn1.find_next('span').get_text())
            team = cn0.get_text()
            team_rating = float(cn0.find_next('span').get_text())
        else:
            club = cn0.get_text()
            club_rating = float(cn0.find_next('span').get_text())
            team = cn1.get_text()
            team_rating = float(cn1.find_next('span').get_text())
    else:
        if cn0.get_text() == tname:
            club = None
            club_rating = np.nan
            team = cn0.get_text()
            team_rating = float(cn0.find_next('span').get_text())
        else:
            club = cn0.get_text()
            club_rating = float(cn0.find_next('span').get_text())
            team = tname
            team_rating = np.nan
            
    # player statistics
    r1 = html_player.find('div', attrs={'class': 'mt-2 mb-2'})
    r1_dat = r1.find_all('span', attrs={'class': re.compile('^label.+')})
    r2_dat = (r1.find_next('div', attrs={'class': 'mb-2'})
              .find_all('span', attrs={'class': re.compile('^label.+')}))
            
    stats = [val for val in r1_dat]+[val for val in r2_dat]
            
    return ID, name, full_name, position, age, overall, potential, value, wage, club, club_rating, team, team_rating, stats
 

In [62]:
summary = {}
for date, dhref in tqdm.tqdm(dates_href.items()):
    for team, thref in full_teams_href.items():
        for player, phref in get_players_href(thref, dhref).items():
            ID, name, full_name, position, age, overall, potential, value, wage, club, club_rating, team, team_rating, stats = get_players(phref, dhref)
            summary[(ID, date)] = {
                'name': name,
                'full_name': full_name,
                'position': position,
                'age': age,
                'overall': overall,
                'potential': potential,
                'value': value,
                'wage': wage,
                'club': club,
                'club_rating': club_rating,
                'team': team,
                'team_rating': team_rating,
                'stats': stats
            }


  0%|                                                                                          | 0/432 [00:00<?, ?it/s]


NameError: name 'tname' is not defined